In [1]:
# OceanRAIN reading, first attempt mid Nov 2017.
#  initially thought of running ARTS with OR PSD...
# revisited 19/3/18
# work on again late may / early june, producing some plots for 
#  earthcare workshop poster

import h5py
import glob
#import numpy as np
#import matplotlib.pyplot as plt
import subprocess


/home/dudavid/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
dire = '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/DSD/'
#fi = 'OceanRAIN__W__RV-SonneI_DFCG__UHAM-ICDC__v1_0.nc' #one of smaller W files
# W = Water cycle (temporally continuous, contains non-raining time)
# M = Microphysics (just rain, discontinuous, derived DSD fields)
# R = Raw microphysics (like M but with raw data)
wfiles = glob.glob(dire+'*_W_*.nc')
print(wfiles)

['/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/DSD/OceanRAIN__W__MS-The-World_C6RW4__UHAM-ICDC__v1_0.nc', '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/DSD/OceanRAIN__W__RV-Investigator_VLMJ__UHAM-ICDC__v1_0.nc', '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/DSD/OceanRAIN__W__RV-Maria-S-Merian_DBBT__UHAM-ICDC__v1_0.nc', '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/DSD/OceanRAIN__W__RV-Meteor_DBBH__UHAM-ICDC__v1_0.nc', '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/DSD/OceanRAIN__W__RV-Polarstern_DBLK__UHAM-ICDC__v1_0.nc', '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/DSD/OceanRAIN__W__RV-Roger-Revelle_KAOU__UHAM-ICDC__v1_0.nc', '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/DSD/OceanRAIN__W__RV-SonneI_DFCG__UHAM-ICDC__v1_0.nc', '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/DSD/OceanRAIN__W__RV-SonneII_DBBE__UHAM-ICDC__v1_0.nc']


In [3]:
# set up parameters for grid down below
gsize = 2.0 # grid size in degrees
startlat = 90.0 # going to poles
nx, ny = int(360.0/gsize), int(2*startlat/gsize)
grid, sgrid, cgrid = np.zeros([ny,nx]), np.zeros([ny,nx]), np.zeros([ny,nx]) # strat, conv
an, sn, cn = np.zeros([ny,nx]), np.zeros([ny,nx]), np.zeros([ny,nx]) # counts



In [4]:

muall,dmall,Noall,csall,ssall,dtall=[],[],[],[],[],[]
laall,rgall,rdall=[],[],[]
reflC,reflU,reflA=[],[],[] # reflectivity at C-,Ku-,Ka-bands

for fi in wfiles[:]:

    print(fi)
    f = h5py.File(fi)

    pf = np.array(f['precip_flag']) # 1 (there are 2)
    # 0=rain, 1=snow, 2=mixedphase, 3=noprecip(true-zero), 4/5=nodata, 9=misval
    # ONLY WANT RAINING DATA POINTS, SO SUBSET ALL OTHER VARS USING PRECIP_FLAG 
    pop= np.array(f['probability_for_rain'])
    rain = np.logical_and(pf==0, pop==1.0)
    #print(info(pop[rain]))
    #print(info(pf[pop == 1.0]))
    
    #t  = np.array(f['air_temperature'])[rain] # in C
    dt = np.array(f['dew_point_temperature'])[rain] # in C
    csi= np.array(f['convective_stratiform_index'])[rain] # 0 strat, 1 conv
    #c  = np.array(f['ceiling'])
    #jd = np.array(f['julian_date'])[rain] # days since jan 1 1994
    la = np.array(f['latitude'])[rain] # deg N
    lo = np.array(f['longitude'])[rain] # deg E
    ss = np.array(f['sea_surface_temperature'])[rain] # in C
    #wst= np.array(f['true_wind_speed']) # m/s
    w10 = np.array(f['wind_speed_in_10m_height'])[rain] # m/s
    rrd= np.array(f['theoretical_rain_rate_disdrometer'])[rain] # mm/h
    #rrg= np.array(f['rain_gauge_precipitation_rate']) # mm/h
    #rro= np.array(f['ODM470_precipitation_rate_R']) # mm/h --identical to theor. RR (above) if raining
    #dsd params:
    mu = np.array(f['shape_parameter_of_normalized_gamma'])[rain] # [dimensionless]
    dm = np.array(f['mass_weighted_mean_diameter_of_normalized_gamma'])[rain] # [mm]
    No = np.array(f['intercept_of_normalized_gamma'])[rain] # [mm-1 m-3]
    RC = f['C_band_reflectivity'][:][rain]
    RU = f['Ku_band_reflectivity'][:][rain]
    RA = f['Ka_band_reflectivity'][:][rain]
    # From Testud et al. (2004): N(D)=No*F(D/Dm) 
    #  where F(x)=const*((4+mu)^(4+mu))/gamma(4+mu) * x^mu * exp(-(4+mu)x)
    # can calculate LWC by integrating over N(D)*D^3*dD
    
    # save to big ole arrays for analysis below on full dataset
    use = dm>0
    muall.extend(mu[use])
    dmall.extend(dm[use])
    Noall.extend(No[use])
    #rgall.extend(rrg[use]) # using disdrometer or gauge?? (rrd/rrg, see above)
    rdall.extend(rrd[use]) # using disdrometer or gauge?? (rrd/rrg, see above)
    #roall.extend(rrd[use]) #  or odm (rrd/rrg, see above)
    csall.extend(csi[use])
    ssall.extend(ss[use])
    dtall.extend(dt[use])
    laall.extend(la[use])
    reflC.extend(RC[use])
    reflU.extend(RU[use])
    reflA.extend(RA[use])

    # make grid, fill it in with data from each ship
    ladex = np.round((la[:]+startlat-gsize*.5) / gsize)
    lodex = np.round((lo[:]+180.0-gsize*.5) / gsize)
    # extra subsetting for real values:
    fld = mu #dm #No #mu #rrd
    rf = -4 # if ^ is mu, make less than zero!
    afld,aladex,alodex,cs = fld[fld>rf],ladex[fld>rf],lodex[fld>rf],csi[fld>rf]
    #print(info(afld))
     #no missing data if dsd params? conv/strat flag -9 for missing
    cfld,cladex,clodex = afld[cs==1], aladex[cs==1], alodex[cs==1]
    sfld,sladex,slodex = afld[cs==0], aladex[cs==0], alodex[cs==0]
    for x in range(nx):
        if np.any(lodex == x):
            dexsub = np.where(alodex == x)
            subset = afld[dexsub] 
            subsetla = aladex[dexsub]
            for y in range(ny):
                if np.any(subsetla == y):
                    lolasubset = np.where(subsetla == y)
                    grid[y,x] += subset[lolasubset].sum() # add up, get mean afterward
                    an[y,x] += len(subset[lolasubset])
    for x in range(nx):  # strat loops
        if np.any(slodex == x):
            dexsub = np.where(slodex == x)
            subset = sfld[dexsub] 
            subsetla = sladex[dexsub]
            for y in range(ny):
                if np.any(subsetla == y):
                    lolasubset = np.where(subsetla == y)
                    sgrid[y,x] += subset[lolasubset].sum() # add up, get mean afterward
                    sn[y,x] += len(subset[lolasubset])
    for x in range(nx):  # conv loops
        if np.any(clodex == x):
            dexsub = np.where(clodex == x)
            subset = cfld[dexsub] 
            subsetla = cladex[dexsub]
            for y in range(ny):
                if np.any(subsetla == y):
                    lolasubset = np.where(subsetla == y)
                    cgrid[y,x] += subset[lolasubset].sum() # add up, get mean afterward
                    cn[y,x] += len(subset[lolasubset])

In [6]:
# look at statistics for dsd fields from all ships together
csa = np.array(csall)
st = csa==0
cv = csa==1

mua = np.array(muall)
dma = np.array(dmall)
Noa = np.array(Noall)
ssa = np.array(ssall)
#rga = np.array(rgall)
rda = np.array(rdall)
#dta = np.array(dtall)
laa = np.array(laall)
Ku = np.array(reflU)
Ka = np.array(reflA)
C = np.array(reflC)
muac = np.array(muall)[cv]
dmac = np.array(dmall)[cv]
Noac = np.array(Noall)[cv]
ssac = np.array(ssall)[cv]
#rgac = np.array(rgall)[cv]
rdac = np.array(rdall)[cv]
#dtac = np.array(dtall)[cv]
laac = np.array(laall)[cv]
muas = np.array(muall)[st]
dmas = np.array(dmall)[st]
Noas = np.array(Noall)[st]
ssas = np.array(ssall)[st]
#rgas = np.array(rgall)[st]
rdas = np.array(rdall)[st]
#dtas = np.array(dtall)[st]
laas = np.array(laall)[st]

# calculate RWC/LWC from Dm and No (No= (4^4)/(pi*rho)*(LWC/Dm^4)  [g m^-3]
rwca= (Noa*1e-6)*dma**4 * math.pi*1000.0/(4.0**4.0) #multiply No by 1e-6 to offset rwc in g/m^3
rwcas = rwca[st]
rwcac = rwca[cv]
#realbig = rwca > 10.0
#print(info(rda[realbig]),info(dma[realbig]),info(C[realbig]))
print('rwc ',np.shape(rwca))
print('rwc ',info(rwca))

print('mu ',info(mua))
print('dm ',info(dma))
print('No ',info(Noa)) # to compare to published No [m^-4] multiply by 1000
print('')
print('mu (strat) ',info(mua[st]))
print('mu (conv) ',info(mua[cv]))
print('')
print('dm (strat) ',info(dma[st]))
print('dm (conv) ',info(dma[cv]))
print('')
print('No (strat) ',info(Noa[st]))
print('No (conv) ' ,info(Noa[cv]))
print('')
print('Ku (strat) ',info(Ku[st]))
print('Ku (conv) ' ,info(Ku[cv]))
print('')
print('Ka (strat) ',info(Ka[st]))
print('Ka (conv) ' ,info(Ka[cv]))
print('')
print('C (strat) ',info(C[st]))
print('C (conv) ' ,info(C[cv]))

rwc  (0,)


ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:
# calculate binned rain rate ranges for given 2D reflectivity spaces
from dmap import grdplt
bsi = .5 # dBZ bin size
xmi, xma, ymi, yma = 10, 60, 10, 50
nx,ny = int((xma-xmi)/bsi), int((yma-ymi)/bsi)
bind, binm = np.zeros([nx,ny]), np.zeros([nx,ny])
bind2, binn = np.zeros([nx,ny]), np.zeros([nx,ny])
ax, ay = C, Ka # setup x/y vars here
print(info(ax),info(ay))
for x in range(nx):
    wharex = np.logical_and(ax >= x*bsi+xmi, ax < (x+1)*bsi+xmi)
    if np.any(wharex):
        for y in range(ny):
            wharey = np.logical_and(ay[wharex] >= y*bsi+ymi, ay[wharex] < (y+1)*bsi+ymi)
            if np.any(wharey):
                bind[x,y] = np.std( rda[wharex][wharey] )
                binm[x,y] = np.mean(rda[wharex][wharey] )
                binn[x,y] = np.size(rda[wharex][wharey] )
                bind2[x,y]= np.std( rda[wharex][wharey] ) / np.mean(rda[wharex][wharey])
                #if bind2[x,y] > 1: print(binn[x,y],info(rda[wharex][wharey]))

print(info(bind))
#grdplt(bind, xmi,xma,ymi,yma, .1,10,1)#, 'Ku vs. Ka Std RR','RR [mm/hr]') # x dims, y dims, v min/max, log flag
#bind[binm>0] /= binm[binm>0]                
#print(info(bind))
bind2 *= 100.0
xla = 'C-band [dBZ]'
yla = r'$K_a$-band [dBZ]'
#figr, axrs = plt.subplots(nrows=1, ncols=2, figsize=(9, 5))
#fig1 = plt.figure(figsize=[6,6])
##axrs[0] = grdplt(binm, xmi,xma,ymi,yma, .05,50,1, 6,5,'',r'Mean RR $[mm hr^{-1}]$',xla,yla)
#grdplt(binm, xmi,xma,ymi,yma, .05,50,1, 6,5,'',r'Mean RR $[mm hr^{-1}]$',xla,yla)
#fig1.savefig('img/refl1.png',transparent=True,bbox_inches="tight",dpi=300)
##grdplt(bind, xmi,xma,ymi,yma, .1,20,1, 6,5, 'Ka vs. C Std RR','Std') # x dims, y dims, v min/max, log flag
#fig2 = plt.figure(figsize=[6,6])
#grdplt(bind2, xmi,xma,ymi,yma, 0,80,0, 6,5, '',r'$\sigma$(RR) / Mean(RR) [%]',xla,yla) # x dims, y dims, v min/max, log flag
#fig2.savefig('img/refl2.png',transparent=True,bbox_inches="tight",dpi=300)
##grdplt(binn, xmi,xma,ymi,yma, 1,5e3,1, 6,5, 'Ka vs. C Counts','#') # x dims, y dims, v min/max, log flag

In [ ]:
#
#crealz = np.where(cn >= 1)
#srealz = np.where(sn >= 1)
#arealz = np.where(an >= 1)
#s_mean,c_mean,a_mean = np.zeros([ny,nx]) - 9.9, np.zeros([ny,nx]) - 9.9, np.zeros([ny,nx]) - 9.9
#s_mean[srealz] = sgrid[srealz] / sn[srealz] # calculate means                
#c_mean[crealz] = cgrid[crealz] / cn[crealz] # calculate means                
#a_mean[arealz] = grid[arealz] / an[arealz] # calculate means                
##print(info(cgrid))
##print(info(s_mean))
##print(cgrid.max())
#print(info(s_mean[srealz]))
#print(info(c_mean[crealz]))
#h,ed = np.histogram(Noa,bins=10)
#print(h)
#print(ed)

In [ ]:
# produce histogram plots of the three DSD variables (and lwc?) separated by 
#  x-tiles of a chosen ancillary variable
import pygal
from pygal.style import DarkColorizedStyle,DarkSolarizedStyle,CleanStyle
from decimal import Decimal # display No in sci notation

xtiles = 8
varese = ['SST','RR','Nw','Mu','Dm'] # make sure this matches list of arrays below 
#varese = ['No','Dm','mu','RR','SST']  # added rwc on the end below
#varese = ['No','Lat','Td','RR','SST']
vs = [[ssa,rda,Noa,mua,dma,rwca],[ssac,rdac,Noac,muac,dmac,rwcac],[ssas,rdas,Noas,muas,dmas,rwcas]]
#print(np.shape(vs[1][1]))
#scs = 'str'
scss = ['both','con','str']

##for vart in varese:
#for v in range(len(varese)):
#  for sc in range(3):
#    scs = scss[sc]
#    if scs=='both': subset = np.logical_and(ssa>0,dma>0)
#    if scs=='con': subset = np.logical_and(ssac>0,dmac>0)
#    if scs=='str': subset = np.logical_and(ssas>0,dmas>0)
#    var = np.array(vs[sc][v])[subset]
#    vart=varese[v]
#    print(v,sc,vart,scs)
#    dexs = np.argsort(var)
#    nv = np.size(var) #  # elements
#
#    mbinz = [-3,-2,-1,-.5,0,.5,1,1.5,2,3,5,7,10,15]
#    dbinz = [.4,.6,.8,1,1.2,1.4,1.6,1.8,2,2.2,2.4,2.7,3,3.5,4,5]
#    Nbinz = [10**(x*.2+3) for x in range(11)]
#    rbinz = [.005,.01,.02,.04,.07,.1,.125,.15,.18,.21,.25,.3,.4,.6,1,4]
#    rrbinz= [.01,.05,.1,.2,.4,.7,1,1.5,2,3,5,7.5,10,15,20,30,50]
#    mh = np.zeros([xtiles,len(mbinz)-1])
#    dh = np.zeros([xtiles,len(dbinz)-1])
#    rh = np.zeros([xtiles,len(rbinz)-1])
#    rrh= np.zeros([xtiles,len(rrbinz)-1])
#    Nh = np.zeros([xtiles,len(Nbinz)-1])
#    btmp,tpp = [],[]
#
#    for i in range(xtiles):
#        p1,p2 = int(i*nv/xtiles),int((i+1)*nv/xtiles)
#        btmp.append("%.2f" % round(var[dexs[p1]],1))
#        tpp.append("%.1f" % round(var[dexs[p2-1]],1))
#        #tpp.append('')
#        #print(info(var[dexs[p1:p2]]))
#        hm, medges  = np.histogram(vs[sc][3][subset][dexs[p1:p2]],bins=mbinz) #mu
#        hd, dedges  = np.histogram(vs[sc][4][subset][dexs[p1:p2]],bins=dbinz) #dm
#        hN, Nedges  = np.histogram(vs[sc][2][subset][dexs[p1:p2]],bins=Nbinz) #Nw
#        hr, redges  = np.histogram(vs[sc][5][subset][dexs[p1:p2]],bins=rbinz) #rwc
#        hrr,rredges = np.histogram(vs[sc][1][subset][dexs[p1:p2]],bins=rrbinz) # RR #same down here
#        mh[i,:] = hm
#        dh[i,:] = hd
#        Nh[i,:] = hN
#        rh[i,:] = hr
#        rrh[i,:]= hrr
#
#    #print(hm,hd,hN,hr,hrr)
#    mb = [str(i) for i in mbinz]
#    db = [str(i) for i in dbinz]
#    rb = [str(i) for i in rbinz]
#    rrb= [str(i) for i in rrbinz]
#    Nb = ["{:.2E}".format(Decimal(i)) for i in Nbinz]
#
#    lc1 = pygal.Line(title='Mu/'+vart+'.'+scs,x_title='Mu bins',y_title='Counts',x_labels=mb,\
#                    style=DarkSolarizedStyle,legend_at_bottom=True)
#                    #style=CleanStyle,legend_at_bottom=True)
#    for i in range(xtiles): lc1.add(vart+': '+str(btmp[i])+'- '+str(tpp[i]), mh[i,:])
#    fi1 = 'img/'+vart+'.'+scs+'_Mu'
#    lc1.render_to_file(fi1+'.svg')
#
#    lc2 = pygal.Line(title='Dm/'+vart+'.'+scs,x_title='Dm bins',y_title='Counts',x_labels=db,\
#                    style=DarkSolarizedStyle,legend_at_bottom=True)
#    for i in range(xtiles): lc2.add(vart+': '+str(btmp[i])+'- '+str(tpp[i]), dh[i,:])
#    fi2 = 'img/'+vart+'.'+scs+'_Dm'
#    lc2.render_to_file(fi2+'.svg')
#
#    #lc3 = pygal.Line(title='Nw/'+vart,x_title=r'Nw bins $[mm^{-1} m^{-3}]$',y_title='Counts',x_labels=Nb,\
#    lc3 = pygal.Line(title='Nw/'+vart+'.'+scs,x_title='Nw bins',y_title='Counts',x_labels=Nb,\
#                    style=DarkSolarizedStyle,logarithmic=True,legend_at_bottom=True)
#    for i in range(xtiles): lc3.add(vart+': '+str(btmp[i])+'- '+str(tpp[i]), Nh[i,:])
#    fi3 = 'img/'+vart+'.'+scs+'_Nw'
#    lc3.render_to_file(fi3+'.svg')
#
#    lc4 = pygal.Line(title='RWC/'+vart+'.'+scs,x_title='RWC bins',y_title='Counts',x_labels=rb,\
#                    style=DarkSolarizedStyle,logarithmic=True,legend_at_bottom=True)
#    for i in range(xtiles): lc4.add(vart+': '+str(btmp[i])+'- '+str(tpp[i]), rh[i,:])
#    fi4 = 'img/'+vart+'.'+scs+'_RWC'
#    lc4.render_to_file(fi4+'.svg')
#
#    lc5 = pygal.Line(title='RR/'+vart+'.'+scs,x_title='RR bins',y_title='Counts',x_labels=rrb,\
#                    style=DarkSolarizedStyle,logarithmic=True,legend_at_bottom=True)
#    for i in range(xtiles): lc5.add(vart+': '+str(btmp[i])+'- '+str(tpp[i]), rrh[i,:])
#    fi5 = 'img/'+vart+'.'+scs+'_RR'
#    lc5.render_to_file(fi5+'.svg')
#    
#    subprocess.call(['convert','-density','180',fi1+'.svg',fi1+'.png'])
#    subprocess.call(['convert','-density','180',fi2+'.svg',fi2+'.png'])
#    subprocess.call(['convert','-density','180',fi3+'.svg',fi3+'.png'])
#    subprocess.call(['convert','-density','180',fi4+'.svg',fi4+'.png'])
#    subprocess.call(['convert','-density','180',fi5+'.svg',fi5+'.png'])


In [ ]:
subsette = [rda > 0.0]
v1 = np.log10(Noa)[subsette]
#v1 = Noa[subsette]
v2 = dma[subsette]
v3 = mua[subsette]
v4 = rda[subsette]
#v4 = np.log10(rda)[subsette]
v5 = rwca[subsette]
#print(info(v1),info(v2),info(v3),info(v4),info(v5))
#rning = [rga > 0]
#  rga (gauge rain rate) has many missing and suspiciously similar values, sticking to disdrometer RRs for now
#print(rda[rning][0:300])
#print(rga[rning][0:300])
#print(np.corrcoef(rda[rning],rga[rning])[1,0])
cors1 = [np.corrcoef(v1,v2)[1,0],np.corrcoef(v1,v3)[1,0],np.corrcoef(v1,v4)[1,0],\
         np.corrcoef(v1,v5)[1,0]]#, np.corrcoef(v1,ssa)[1,0] ]
cors2 = [np.corrcoef(v2,v1)[1,0],np.corrcoef(v2,v3)[1,0],np.corrcoef(v2,v4)[1,0],\
         np.corrcoef(v2,v5)[1,0]]#, np.corrcoef(v2,ssa)[1,0] ]
cors3 = [np.corrcoef(v3,v1)[1,0],np.corrcoef(v3,v2)[1,0],np.corrcoef(v3,v4)[1,0],\
         np.corrcoef(v3,v5)[1,0]]#, np.corrcoef(v3,ssa)[1,0] ]
cors4 = [np.corrcoef(v4,v1)[1,0],np.corrcoef(v4,v2)[1,0],np.corrcoef(v3,v4)[1,0],\
         np.corrcoef(v4,v5)[1,0]]#, np.corrcoef(v3,ssa)[1,0] ]
print(cors1)
print(cors2)
print(cors3)
print(cors4)

subsette = [rda > 0.0]
v1 = np.log10(Noa)[subsette]
#v1 = Noa[subsette]
v2 = dma[subsette]
v3 = mua[subsette]
v4 = rda[subsette]
#v4 = np.log10(rda)[subsette]
v5 = rwca[subsette]
cors1 = [np.corrcoef(v1,v2)[1,0],np.corrcoef(v1,v3)[1,0],np.corrcoef(v1,v4)[1,0],\
         np.corrcoef(v1,v5)[1,0]]#, np.corrcoef(v1,ssa)[1,0] ]
cors2 = [np.corrcoef(v2,v1)[1,0],np.corrcoef(v2,v3)[1,0],np.corrcoef(v2,v4)[1,0],\
         np.corrcoef(v2,v5)[1,0]]#, np.corrcoef(v2,ssa)[1,0] ]
cors3 = [np.corrcoef(v3,v1)[1,0],np.corrcoef(v3,v2)[1,0],np.corrcoef(v3,v4)[1,0],\
         np.corrcoef(v3,v5)[1,0]]#, np.corrcoef(v3,ssa)[1,0] ]
cors4 = [np.corrcoef(v4,v1)[1,0],np.corrcoef(v4,v2)[1,0],np.corrcoef(v3,v4)[1,0],\
         np.corrcoef(v4,v5)[1,0]]#, np.corrcoef(v3,ssa)[1,0] ]
print(cors1)
print(cors2)
print(cors3)
print(cors4)

In [ ]:
#
#p1 = plt.figure(figsize=[8,7])
##plt.plot(mbinz[1:],mh[0,:],'k')#,xaxis='mu',yaxis='counts')
#for i in range(xtiles):
#    plt.plot(mbinz[1:],mh[i,:],'k' )#style=DarkColorizedStyle)
#plt.xlabel('mu'),plt.ylabel('counts')
#
#p2 = plt.figure(figsize=[8,7])
#for i in range(xtiles):
#    plt.plot(dbinz[1:],dh[i,:],'k')
#plt.xlabel('dm'),plt.ylabel('counts')
#
#p3 = plt.figure(figsize=[8,7])
#for i in range(xtiles):
#    plt.plot(Nbinz[1:],Nh[i,:],'k')
#plt.xlabel('No'),plt.ylabel('counts')
#plt.xscale('log')
#
#p4 = plt.figure(figsize=[8,7])
#for i in range(xtiles):
#    plt.plot(rbinz[1:],rh[i,:],'k')
#plt.xlabel('RWC'),plt.ylabel('counts')
#plt.xscale('log')
#
#plt.show()

In [ ]:
#fig1 = plt.figure(figsize=[16,11])
#mapz = grdmap(sn,-180,-startlat,1,cn.max(),1,'stra counts')
#fig2 = plt.figure(figsize=[16,11])
#mapz = grdmap(cn,-180,-startlat,1,cn.max(),1,'conv counts')

#fig3 = plt.figure(figsize=[16,11])
#mapz = grdmap(an,-180,-startlat,1,an.max()/2.0,1,'All rain definite counts')


In [ ]:
#vmi,vma = -2, 15 #a_mean.min(),a_mean.max()
#xp = 0 #1 # plt in exponential or not
#tit = 'mu'#'Dm'
##fig = plt.figure(figsize=[16,11])
##mapz = grdmap(a_mean,-180,-startlat,vmi,vma,xp,'all '+tit)
#fig = plt.figure(figsize=[16,11])
#mapz = grdmap(s_mean,-180,-startlat,vmi,vma,xp,'stra '+tit)
#fig = plt.figure(figsize=[16,11])
#mapz = grdmap(c_mean,-180,-startlat,vmi,vma,xp,'conv '+tit)
#


In [ ]:
print(cn.sum())
print(sn.sum())
print(an.sum())

In [ ]:
# create latitudinal box-and-whisker plots for Dm and mu (perhaps weighted by RR/RWC?)
bznss = 20 # degrees lat
maxlat, minlat = 80, -60
ny = int((maxlat - minlat)/bznss)
print(ny)
latz = np.array(laall)
zave = np.zeros(ny)
for y in range(ny):
    lasub = np.logical_and( latz >= minlat+bznss*y, latz < minlat+(bznss*(y+1)))
    print(np.size(latz[lasub]),minlat+bznss*y,minlat+bznss*(y+1))
    zave[y] = np.size(latz[lasub])
multiple = np.round(zave/float(np.min(zave))).astype(int)
print(multiple)

subr = Ka > 18 # subset of rain rates for boxplot
#subr = rda < 0.5 # subset of rain rates for boxplot

zima = 2660 #np.min(zave).astype(int)
if bznss==20: zima=5000
zmu, zdm, znw = np.zeros([zima,ny])+np.nan, np.zeros([zima,ny])+np.nan, np.zeros([zima,ny])+np.nan
zcmu, zcdm, zcnw = np.zeros([zima,ny])+np.nan, np.zeros([zima,ny])+np.nan, np.zeros([zima,ny])+np.nan
for y in range(ny):
    lasub = np.logical_and( latz >= minlat+bznss*y, latz < minlat+(bznss*(y+1)))
    print(y, ny, len(dma[lasub]))
    lenny = len(dma[lasub])
    if lenny > zima: lenny=zima
    zdm[0:lenny,y] = dma[lasub][0:lenny]
    zmu[0:lenny,y] = mua[lasub][0:lenny]
    znw[0:lenny,y] = Noa[lasub][0:lenny]
    #zdm[:,y] = dma[lasub][::multiple[y]][0:zima]
    #zmu[:,y] = mua[lasub][::multiple[y]][0:zima]
    #znw[:,y] = Noa[lasub][::multiple[y]][0:zima]
    lasub = np.logical_and( latz[subr] >= minlat+bznss*y, latz[subr] < minlat+(bznss*(y+1)))
    #print(y, ny, len(dma[subr][lasub]))
    lenny = len(dma[subr][lasub])
    if lenny > zima: lenny=zima
    zcdm[0:lenny,y] = dma[subr][lasub][0:lenny]
    zcmu[0:lenny,y] = mua[subr][lasub][0:lenny]
    zcnw[0:lenny,y] = Noa[subr][lasub][0:lenny]
    
mask = ~np.isnan(zdm)
fdm = [d[m] for d, m in zip(zdm.T, mask.T)]
fmu = [d[m] for d, m in zip(zmu.T, mask.T)]
fnw = [d[m] for d, m in zip(znw.T, mask.T)]
mask = ~np.isnan(zcdm)
fcdm = [d[m] for d, m in zip(zcdm.T, mask.T)]
fcmu = [d[m] for d, m in zip(zcmu.T, mask.T)]
fcnw = [d[m] for d, m in zip(zcnw.T, mask.T)]

zex = np.arange(minlat,maxlat+bznss,bznss)#+bznss/2.0
lb = [str(i) for i in zex]
ll,ul = 5, 95 # lower and upper limits for whiskers
fs = 14 # fontsize

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(9, 12))
axes[0].boxplot(fdm, sym='',vert=False, whis=[ll,ul])
axes[1].boxplot(fmu, sym='',vert=False, whis=[ll,ul])
axes[2].boxplot(fnw, sym='',vert=False, whis=[ll,ul])
axes[0].set_xlabel(r"$D_m [mm]$",fontsize=fs)
axes[1].set_xlabel(r"$\mu$",fontsize=fs)
axes[2].set_xlabel(r"$N_W [mm^{-1} m^{-3}]$",fontsize=fs)
axes[0].set_ylabel(r'Latitude $[^0 N]$',fontsize=fs)
axes[1].set_ylabel(r'Latitude $[^0 N]$',fontsize=fs)
axes[2].set_ylabel(r'Latitude $[^0 N]$',fontsize=fs)
axes[0].set_yticks(np.arange(ny/2+1)*2+.5)
axes[1].set_yticks(np.arange(ny/2+1)*2+.5)
axes[2].set_yticks(np.arange(ny/2+1)*2+.5)
axes[0].set_yticklabels(lb[::2])
axes[1].set_yticklabels(lb[::2])
axes[2].set_yticklabels(lb[::2])
axes[2].set_xscale('log')
axes[0].set_xlim([0.5,2.5])
axes[1].set_xlim([-3,10])
axes[2].set_xlim([2e2,1e5])
axes[0].set_ylim([0,ny+1])
axes[1].set_ylim([0,ny+1])
axes[2].set_ylim([0,ny+1])

#figc, axcs = plt.subplots(nrows=3, ncols=1, figsize=(7, 12))
#axes[0,1].boxplot(fcdm, sym='',vert=False, whis=[ll,ul])
#axes[1,1].boxplot(fcmu, sym='',vert=False, whis=[ll,ul])
#axes[2,1].boxplot(fcnw, sym='',vert=False, whis=[ll,ul])
#axes[0,1].set_xlabel(r"$D_m [mm]$")
#axes[1,1].set_xlabel(r"$\mu$")
#axes[2,1].set_xlabel(r"$N_0^* [mm^{-1} m^{-3}]$")
#axes[0,1].set_ylabel(r'Latitude $[^0 N]$')
#axes[1,1].set_ylabel(r'Latitude $[^0 N]$')
#axes[2,1].set_ylabel(r'Latitude $[^0 N]$')
#axes[0,1].set_yticks(np.arange(ny+1)+.5)
#axes[1,1].set_yticks(np.arange(ny+1)+.5)
#axes[2,1].set_yticks(np.arange(ny+1)+.5)
#axes[0,1].set_yticklabels(lb)
#axes[1,1].set_yticklabels(lb)
#axes[2,1].set_yticklabels(lb)
#axes[2,1].set_xscale('log')
#axes[0,1].set_xlim([0.5,2.5])
#axes[1,1].set_xlim([-3,12])
#axes[2,1].set_xlim([5e2,1e5])

In [ ]:
# the histograms being passed to GPM_CMB.ipynb are the latitude sub-selected arrays
#  defined above, to yield even latitudinal band counts

binz = np.append(np.arange(0.1,3.0,0.1),[4.0,10.0]) # '10' is for upper bound, since gpm data the 30 bins are 
#   defined as <= to the bin above them, so last bin is 4mm+
# calculate and output normalized histograms of Dm for each lat band:
nhistdm = np.zeros([30,7])
print(np.shape(fdm))
for y in range(7):
    #print(np.histogram(np.asarray(fdm)[y,:],bins=binz))
    nhistdm[:,y] = np.histogram(np.asarray(fdm)[y,:],bins=binz)[0]
    nhistdm[:,y] /= np.sum(nhistdm[:,y]) # normalize
print(info(nhistdm))
print(nhistdm[:,3])
np.save('OR_7lat_Dm.npy', nhistdm)

In [ ]:
# unitless, for n(D) in m^-4
nwout = np.log10( np.array(fnw) * 1000.0 )
nbnz = np.array([4.0,    4.2,    4.4,    4.6,    4.8,    5.0,     \
              5.2,    5.4,    5.6,    5.8,    6.0,    6.2,     \
              6.4,    6.6,    6.8,    7.0,    7.2,    7.4,     \
              7.6,    7.8,    8.0,    8.2,    8.4,    8.6,  \
              8.8,    9.0,    9.2,    9.4,    9.6,    10.0, 11 ])
#   defined as <= to the bin above them
# calculate and output normalized histograms of Nw for each lat band:
print(info(fnw*1000))
print(info(nwout)) # OR data in mm^-1 m^-3, so *1000 to get m^-4
nhistnw = np.zeros([30,7])
#print(np.shape(fnw))
for y in range(7):
    nhistnw[:,y] = np.histogram(nwout[y,:],bins=nbnz)[0]
    nhistnw[:,y] /= np.sum(nhistnw[:,y]) # normalize
print(info(nhistnw))
print(nhistnw[:,3])
np.save('OR_7lat_Nw.npy', nhistnw)

In [ ]:
#finam = 'img/whiskers.v3.png'
#plt.savefig(finam,bbox_inches="tight",transparent=True,dpi=300)
#plt.show()